In [1]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib 
from dateutil.relativedelta import relativedelta
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np

from Tool_v1 import *


# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
yf.pdr_override()

In [2]:
stock_name=["AAPL","GOOG","AMZN","TSLA","BABA","PTPI","BTCY",'FOX',"NVDA","DIS","NKE","XOM","KO",
            "2319.HK","0700.HK","1211.HK","6133.hk","0567.hk",'2318.HK','1448.HK','1810.HK',"3690.HK","2018.HK"]
start="2020-12-31"
end="2022-12-31"

for i in stock_name:
    #MACD619RSI21_8020
    stock_data=pct_change(i,start,end)
    stock_data=stock_data.copy()
    #指标运算
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI_MACD(stock_data,21,80,20)
    stock_data= my_rsi(stock_data, 21)
    stock_data = stock_data.dropna()
    
    #lr = LogisticRegression()
    # 设置正则化类型（L1, L2, Elastic Net）
    penalty = 'l2' 

    # 设置正则化强度（C）
    C = 0.01
    #logistic regression
    lr = LogisticRegression(penalty =penalty, C=C) 
    
    X = stock_data.loc[:,["re_close", "re_open", "my_rsi", "Position", "RSI21"]].values
    

    st_x= StandardScaler()  
    X= st_x.fit_transform(X)  

    #Y = stock_data.loc[:,["up_or_down"]].values.ravel()
    Y = np.where(stock_data['Open'].shift(-1) > stock_data['Open'],1,-1)
    #Y = np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)

    #X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8)

    split = int(0.8*len(stock_data))
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
    
    lr.fit(X_train, Y_train)
    y_pred = lr.predict(X_test)
    f1_score(y_pred, Y_test)
    
    
    print(i,": ","model_score: ",lr.score(X_test,Y_test)," f1_score: ",f1_score(y_pred, Y_test)," accuary_score: ",accuracy_score(Y_test,y_pred))
    print()

AAPL :  model_score:  0.8144329896907216  f1_score:  0.7906976744186047  accuary_score:  0.8144329896907216

GOOG :  model_score:  0.8144329896907216  f1_score:  0.7857142857142858  accuary_score:  0.8144329896907216

AMZN :  model_score:  0.7628865979381443  f1_score:  0.7294117647058823  accuary_score:  0.7628865979381443

TSLA :  model_score:  0.865979381443299  f1_score:  0.8311688311688312  accuary_score:  0.865979381443299

BABA :  model_score:  0.7216494845360825  f1_score:  0.689655172413793  accuary_score:  0.7216494845360825

PTPI :  model_score:  0.7835051546391752  f1_score:  0.676923076923077  accuary_score:  0.7835051546391752

BTCY :  model_score:  0.845360824742268  f1_score:  0.8148148148148148  accuary_score:  0.845360824742268

FOX :  model_score:  0.8556701030927835  f1_score:  0.8444444444444444  accuary_score:  0.8556701030927835

NVDA :  model_score:  0.8247422680412371  f1_score:  0.8247422680412373  accuary_score:  0.8247422680412371

DIS :  model_score:  0.886

/Users/liangfanxiao/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input contains infinity or a value too large for dtype('float64').